In [5]:
import add_path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

In [6]:
from transformers import AutoTokenizer
from models.BERT.tucoregcn_bert_pytorch import TUCOREGCN_BertForSequenceClassification, TUCOREGCN_BertConfig
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tucore_gcn_bert = TUCOREGCN_BertForSequenceClassification(TUCOREGCN_BertConfig.from_json_file("../models/BERT/tucoregcn_bert_mlc.json"))
tucore_gcn_bert.cuda()

False
False


TUCOREGCN_BertForSequenceClassification(
  (tucoregcn_bert): TUCOREGCN_Bert(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (speaker_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutpu

In [7]:
tucore_gcn_bert.load_state_dict(torch.load("../TUCOREGCN_BERT_DialogRE/tucoregcn_pytorch_model.pt", map_location="cuda"))
tucore_gcn_bert.cuda()

TUCOREGCN_BertForSequenceClassification(
  (tucoregcn_bert): TUCOREGCN_Bert(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (speaker_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutpu

In [8]:
from data import TUCOREGCNDataset, TUCOREGCNDataloader
dev_set = TUCOREGCNDataset(src_file="../datasets/DialogRE", save_file="../datasets/DialogRE/dev_BERT.pkl", max_seq_length=512, tokenizer=tokenizer, n_class=36, encoder_type='BERT')
dev_loader = TUCOREGCNDataloader(dataset=dev_set, batch_size=2, shuffle=False, relation_num=36, max_length=512)
idx, batch = next(enumerate(dev_loader))
entry_idx = 0
input_ids = batch["input_ids"].to("cuda:0")
segment_ids = batch["segment_ids"].to("cuda:0")
input_mask = batch["input_masks"].to("cuda:0")
speaker_ids = batch["speaker_ids"].to("cuda:0")
graphs = batch["graphs"]
mention_id = batch["mention_ids"].to("cuda:0")
turn_mask = batch["turn_masks"].to("cuda:0")

Reading data from ../datasets/DialogRE/dev.json.


FileNotFoundError: [Errno 2] No such file or directory: '../datasets/DialogRE/dev.json/train.json'

In [4]:
output = tucore_gcn_bert(
	input_ids,
	segment_ids,
	input_mask,
	speaker_ids,
	graphs,
	mention_id,
	None,
	turn_mask
)
output

Reading data from ../datasets/DialogRE/dev.json.
load preprocessed data from ../datasets/DialogRE/dev_BERT.pkl.
{'input_ids': tensor([[ 101,    3, 1024,  ...,  102,    3,  102],
        [ 101, 5882, 1015,  ...,  102, 4005,  102]], device='cuda:0'), 'token_type_ids': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]], device='cuda:0'), 'speaker_ids': tensor([[ 0, 12, 12,  ...,  0, 12,  0],
        [ 0,  1,  1,  ...,  0,  0,  0]], device='cuda:0'), 'graphs': [Graph(num_nodes={'node': 23},
      num_edges={('node', 'dialog', 'node'): 40, ('node', 'entity', 'node'): 24, ('node', 'speaker', 'node'): 114},
      metagraph=[('node', 'node', 'dialog'), ('node', 'node', 'entity'), ('node', 'node', 'speaker')]), Graph(num_nodes={'node': 23},
      num_edges={('node', 'dialog', 'node'): 40, ('node', 'entity', 'node'): 10, ('node', 'speaker', 'node'): 114},
      metagraph=[('n

{'graph_output': tensor([[ 0.2171,  0.0617,  0.5196,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.3829, -0.1901,  0.7677,  ...,  0.0000,  0.0000,  0.0000]],
        device='cuda:0', grad_fn=<StackBackward0>),
 'graphs': [Graph(num_nodes={'node': 23},
        num_edges={('node', 'dialog', 'node'): 40, ('node', 'entity', 'node'): 24, ('node', 'speaker', 'node'): 114},
        metagraph=[('node', 'node', 'dialog'), ('node', 'node', 'entity'), ('node', 'node', 'speaker')]),
  Graph(num_nodes={'node': 23},
        num_edges={('node', 'dialog', 'node'): 40, ('node', 'entity', 'node'): 10, ('node', 'speaker', 'node'): 114},
        metagraph=[('node', 'node', 'dialog'), ('node', 'node', 'entity'), ('node', 'node', 'speaker')])],
 'sequence_outputs': tensor([[[ 0.7605, -0.3738, -0.0229,  ...,  0.6189, -0.8695, -1.0398],
          [-0.0275,  0.0611,  0.7089,  ...,  1.0653, -0.9105, -1.2445],
          [-0.3923, -1.0438,  0.3354,  ...,  0.7163, -1.0769, -1.2173],
          ...,
          [ 0.9

In [10]:
tokenizer.pad_token_id

0

In [ ]:
from transformers import Pipeline
from torch import Tensor

class ConversationalTextClassification(Pipeline):
    def _sanitize_parameters(self,*args, **kwargs):
        super().__init__(*args, **kwargs)
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs, maybe_arg=2):
        model_input = Tensor(inputs["input_ids"])
        return {"model_input": model_input}

    def _forward(self, model_inputs):
        # model_inputs == {"model_input": model_input}
        outputs = self.model(**model_inputs)
        # Maybe {"logits": Tensor(...)}
        return outputs

    def postprocess(self, model_outputs):
        best_class = model_outputs["logits"].softmax(-1)
        return best_class

In [5]:
from transformers import pipeline
from transformers.pipelines import PIPELINE_REGISTRY

PIPELINE_REGISTRY.register_pipeline(
    "new-task",
    pipeline_class=MyPipeline,
    pt_model=AutoModelForSequenceClassification,
)

classifier = pipeline("text-classification", model=tucore_gcn_bert, tokenizer=tokenizer, device="cuda:0")

classifier("This movie is disgustingly good !")


The model 'TUCOREGCN_BertForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassificat

{'input_ids': tensor([[  101,  2023,  3185,  2003, 19424,  2135,  2204,   999,   102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'speaker_ids': None, 'graphs': None, 'mention_ids': None, 'turn_mask': None}


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not NoneType

In [8]:
print("\n".join(list(tucore_gcn_bert.state_dict().keys())))

tucoregcn_bert.bert.embeddings.word_embeddings.weight
tucoregcn_bert.bert.embeddings.position_embeddings.weight
tucoregcn_bert.bert.embeddings.token_type_embeddings.weight
tucoregcn_bert.bert.embeddings.speaker_embeddings.weight
tucoregcn_bert.bert.embeddings.LayerNorm.weight
tucoregcn_bert.bert.embeddings.LayerNorm.bias
tucoregcn_bert.bert.encoder.layer.0.attention.self.query.weight
tucoregcn_bert.bert.encoder.layer.0.attention.self.query.bias
tucoregcn_bert.bert.encoder.layer.0.attention.self.key.weight
tucoregcn_bert.bert.encoder.layer.0.attention.self.key.bias
tucoregcn_bert.bert.encoder.layer.0.attention.self.value.weight
tucoregcn_bert.bert.encoder.layer.0.attention.self.value.bias
tucoregcn_bert.bert.encoder.layer.0.attention.output.dense.weight
tucoregcn_bert.bert.encoder.layer.0.attention.output.dense.bias
tucoregcn_bert.bert.encoder.layer.0.attention.output.LayerNorm.weight
tucoregcn_bert.bert.encoder.layer.0.attention.output.LayerNorm.bias
tucoregcn_bert.bert.encoder.layer.0

In [ ]:
import dgl
sequence_outputs = output["sequence_outputs"]
pooled_outputs = output["pooled_outputs"]
mention_ids = batch["mention_ids"].to("cuda:0")
graphs = batch["graphs"]
#initialize some variables
features = None
num_batch_turn = []
slen = input_ids.size(1)
# Iterate over all inputs to be processed
for i in range(len(graphs)):
	sequence_output = sequence_outputs[i]
	mention_id = mention_ids[i]
	pooled_output = pooled_outputs[i]
	# Find the last turn (in the case of tucoregcn, it is the masked speaker dictionary)
	mention_num = torch.max(mention_id)
	# Create a mention matrix idx of mention_num*slen
	num_batch_turn.append(mention_num + 1)
	mention_index = (
		(torch.arange(mention_num) + 1).unsqueeze(1).expand(-1, slen)
	)
	if torch.cuda.is_available():
		mention_index = mention_index.cuda()
	# Create a mentions matrix of slen*mention_num
	mentions = mention_id.unsqueeze(0).expand(mention_num, -1)
	# Generate truth matrix of each speaker's dialogue over the entire conversation
	select_metrix = (mention_index == mentions).float()
	# Factor into the one-hot encoding, the total number of words in each speaker's dialogue
	word_total_numbers = (
		torch.sum(select_metrix, dim=-1).unsqueeze(-1).expand(-1, slen)
	)
	select_metrix = torch.where(
		word_total_numbers > 0,
		select_metrix / word_total_numbers,
		select_metrix,
	)
	# Apply one hot encoding to sequence_output to selectively obtain features
	x = torch.mm(select_metrix, sequence_output)
	x = torch.cat((pooled_output.unsqueeze(0), x), dim=0)
	# Iteratively concatenate features from all sequence_outputs
	if features is None:
		features = x
	else:
		features = torch.cat((features, x), dim=0)
# Batch dgl graphs into 1 graph for efficient computation
graph_big = dgl.batch(graphs)
output_features = [features]

In [115]:
num_batch_turn

[tensor(23, device='cuda:0'), tensor(23, device='cuda:0')]

In [106]:
lstm = tucore_gcn_bert.tucoregcn_bert.LSTM_layers[0]

In [111]:
new_features = []
new_features.append(features[0])
lstm_out = lstm(features[0 + 1 : 0 + idx - 2].unsqueeze(0))
lstm_out

tensor([[-0.0296, -0.1093, -0.0949,  ...,  0.0403, -0.0775,  0.0609],
        [ 0.0269, -0.1400, -0.0430,  ...,  0.0400, -0.0286,  0.0603],
        [ 0.0338, -0.0955, -0.0860,  ...,  0.0678, -0.0762,  0.0849],
        ...,
        [ 0.0447, -0.0806, -0.0882,  ...,  0.0112, -0.0611,  0.0586],
        [-0.0022, -0.0971, -0.0838,  ...,  0.0282, -0.0546,  0.0250],
        [ 0.0282, -0.0570, -0.0888,  ...,  0.0342, -0.0482,  0.0617]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [103]:
output_features

[tensor([[ 4.5792e-01, -1.1729e-02,  3.2297e-01,  ..., -3.7483e-01,
           4.0892e-01, -7.1791e-01],
         [-4.3282e-01, -1.8977e-01,  7.0841e-01,  ...,  7.7599e-01,
          -5.2889e-01, -1.2678e+00],
         [-1.0421e+00, -5.9411e-01,  1.3200e+00,  ...,  2.7433e-01,
          -4.1637e-01, -1.2198e+00],
         ...,
         [ 2.4836e-01, -7.2986e-01,  1.8437e-01,  ...,  6.4670e-01,
          -1.1878e+00, -2.2512e-01],
         [ 1.4898e-01, -6.1494e-01, -5.7474e-01,  ...,  1.6153e+00,
          -2.0723e+00, -3.3880e-01],
         [ 1.2348e+00, -4.7497e-01, -5.3483e-01,  ..., -1.4764e-03,
          -1.1713e+00, -2.1761e-01]], device='cuda:0', grad_fn=<CatBackward0>)]

In [97]:
[4,7,8]+=[2]

SyntaxError: 'list' is an illegal expression for augmented assignment (2151216791.py, line 1)

In [5]:
import dgl
sequence_output = output["sequence_outputs"][0]
pooled_output = output["pooled_outputs"][0]
mention_id = batch["mention_ids"].to("cuda:0")[0]

#initialize some variables
slen = input_ids.size(1)
# Find the last turn (in the case of tucoregcn, it is the masked speaker dictionary)
mention_num = torch.max(mention_id)
# Create a mention matrix idx of mention_num*slen
num_batch_turn = mention_num + 1
mention_index = (
	(torch.arange(mention_num) + 1).unsqueeze(1).expand(-1, slen)
)
if torch.cuda.is_available():
	mention_index = mention_index.cuda()
# Create a mentions matrix of slen*mention_num
mentions = mention_id.unsqueeze(0).expand(mention_num, -1)
# Generate 1 hot encoding of each speaker's dialogue
select_metrix = (mention_index == mentions).float()
# Factor into the one-hot encoding, the total number of words in each speaker's dialogue
word_total_numbers = (
	torch.sum(select_metrix, dim=-1).unsqueeze(-1).expand(-1, slen)
)
select_metrix = torch.where(
	word_total_numbers > 0,
	select_metrix / word_total_numbers,
	select_metrix,
)
# Apply one hot encoding to sequence_output to selectively obtain features
features = torch.mm(select_metrix, sequence_output)
features = torch.cat((pooled_output.unsqueeze(0), features), dim=0)

In [94]:
val, ind = torch.max(torch.LongTensor([[1,2,3],[1,2,4]]),dim=1)
ind

tensor([2, 2])

In [29]:
torch.ones(3,5)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [30]:
mention_nums

tensor([22, 22], device='cuda:0')

tensor([[6, 6, 6, 6, 6, 6, 6, 6],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [8, 8, 8, 8, 8, 8, 8, 8]])

In [95]:
select_metrix

tensor([[0.0000, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000]],
       device='cuda:0')

In [88]:
(torch.arange(5) + 1).unsqueeze(1)

tensor([[1],
        [2],
        [3],
        [4],
        [5]])

In [87]:
temp = torch.LongTensor([6,7,8])
result = torch.arange(0, torch.max(temp)+1).expand(len(temp), -1)
result#>temp.expand(torch.max(temp)+1,-1).T

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8],
        [0, 1, 2, 3, 4, 5, 6, 7, 8],
        [0, 1, 2, 3, 4, 5, 6, 7, 8]])

In [94]:
result.unsqueeze(2)

tensor([[[0],
         [1],
         [2],
         [3],
         [4],
         [5],
         [6],
         [7],
         [8]],

        [[0],
         [1],
         [2],
         [3],
         [4],
         [5],
         [6],
         [7],
         [8]],

        [[0],
         [1],
         [2],
         [3],
         [4],
         [5],
         [6],
         [7],
         [8]]])

In [90]:
result>torch.LongTensor([6,7,8]).unsqueeze(-1)

tensor([[False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False, False]])

In [78]:
mention_index

tensor([[ 1,  1,  1,  ...,  1,  1,  1],
        [ 2,  2,  2,  ...,  2,  2,  2],
        [ 3,  3,  3,  ...,  3,  3,  3],
        ...,
        [20, 20, 20,  ..., 20, 20, 20],
        [21, 21, 21,  ..., 21, 21, 21],
        [22, 22, 22,  ..., 22, 22, 22]], device='cuda:0')

In [76]:
batch["mention_ids"].to("cuda:0")

tensor([[ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0]], device='cuda:0')

In [16]:
sequence_outputs = output["sequence_outputs"]
pooled_outputs = output["pooled_outputs"]
graphs = batch["graphs"]
mention_ids = batch["mention_ids"].to("cuda:0")
mention_ids
#initialize some variables
slen = input_ids.size(1)
# Find the last turn (in the case of tucoregcn, it is the masked speaker dictionary)
mention_nums, _ = torch.max(mention_ids,dim=1)
num_batch_turns = mention_nums + 1
'''
mention_index = (
	(torch.arange(mention_nums,dim=1) + 1).unsqueeze(1).expand(-1, slen)
)
if torch.cuda.is_available():
	mention_index = mention_index.cuda()'''
mentions = mention_ids.unsqueeze(1).expand(-1, mention_num, -1)
mentions

TypeError: arange() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (Number end, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (Number start, Number end, *, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (Number start, Number end, Number step, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [73]:
mention_ids.shape

torch.Size([2, 512])

In [65]:
mention_id.unsqueeze(0).expand(mention_num, -1)

tensor([[ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        ...,
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0]], device='cuda:0')

In [71]:
mention_ids.unsqueeze(1).expand(-1, mention_num, -1)[0]

tensor([[ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        ...,
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0],
        [ 0,  1,  1,  ...,  0, 22,  0]], device='cuda:0')

In [64]:
input_ids.size(1)

512

In [78]:
torch.sum((mention_index==mentions).float(), dim=-1).unsqueeze(-1).expand(-1, slen)

tensor([[ 4.,  4.,  4.,  ...,  4.,  4.,  4.],
        [ 5.,  5.,  5.,  ...,  5.,  5.,  5.],
        [12., 12., 12.,  ..., 12., 12., 12.],
        ...,
        [75., 75., 75.,  ..., 75., 75., 75.],
        [ 2.,  2.,  2.,  ...,  2.,  2.,  2.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]], device='cuda:0')

In [80]:
select_metrix[1]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2000, 0.2000, 0.2000, 0.2000,
        0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [76]:
(mention_index==mentions).float()[1]

tensor([0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [70]:
select_metrix

tensor([[0.0000, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000]],
       device='cuda:0')

In [68]:
mentions.shape

torch.Size([22, 512])

In [69]:
(torch.arange(mention_num) + 1).unsqueeze(1).expand(-1, slen).shape

torch.Size([22, 512])

In [6]:
tokenizer.convert_ids_to_tokens(input_ids[1])

['[CLS]',
 'speaker',
 '1',
 ':',
 'hey',
 '!',
 'speaker',
 '2',
 ':',
 'hey',
 '.',
 'speaker',
 '3',
 ':',
 'hey',
 ',',
 'man',
 '.',
 'what',
 "'",
 's',
 'up',
 '?',
 'speaker',
 '1',
 ':',
 'maybe',
 'you',
 'can',
 'tell',
 'me',
 '.',
 'my',
 'agent',
 'would',
 'like',
 'to',
 'know',
 'why',
 'i',
 'didn',
 "'",
 't',
 'show',
 'up',
 'at',
 'the',
 'audition',
 'i',
 'didn',
 "'",
 't',
 'know',
 'i',
 'had',
 'today',
 '.',
 'the',
 'first',
 'good',
 'thing',
 'she',
 'gets',
 'me',
 'in',
 'weeks',
 '.',
 'how',
 'could',
 'you',
 'not',
 'give',
 'me',
 'the',
 'message',
 '?',
 '!',
 'speaker',
 '3',
 ':',
 'well',
 ',',
 'i',
 "'",
 'll',
 'tell',
 'ya',
 'i',
 'do',
 'enjoy',
 'guilt',
 ',',
 'but',
 ',',
 'ah',
 ',',
 'it',
 'wasn',
 "'",
 't',
 'me',
 '.',
 'speaker',
 '2',
 ':',
 'yes',
 ',',
 'it',
 'was',
 '!',
 'it',
 'was',
 'him',
 '!',
 'uh',
 'huh',
 '!',
 'okay',
 ',',
 'it',
 'was',
 'me',
 '!',
 'speaker',
 '1',
 ':',
 'how',
 'is',
 'it',
 'you',
 '?',


In [5]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[1]))

"[CLS] speaker 1 : hey! speaker 2 : hey. speaker 3 : hey, man. what ' s up? speaker 1 : maybe you can tell me. my agent would like to know why i didn ' t show up at the audition i didn ' t know i had today. the first good thing she gets me in weeks. how could you not give me the message?! speaker 3 : well, i ' ll tell ya i do enjoy guilt, but, ah, it wasn ' t me. speaker 2 : yes, it was! it was him! uh huh! okay, it was me! speaker 1 : how is it you? speaker 2 : well, it was just, it was all so crazy, you know. i mean, chandler was in the closet, counting to 10, and he was up to 7 and i hadn ' t found a place to hide yet. i - i - i meant to tell you, and i wrote it all down on my hand. see, all of it. speaker 1 : yep, that ' s my audition. speaker 4 : see, now this is why i keep notepads everywhere. speaker 2 : yep, and that ' s why we don ' t invite you to play. speaker 5 : what is the great tragedy here? you go get yourself another appointment. speaker 1 : well, estelle tried, you kn

In [57]:
turn_mask[0]

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]], device='cuda:0')

In [52]:
mention_id

tensor([ 0,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
         7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 

In [43]:
mentions.shape

torch.Size([22, 512])

In [38]:
mention_id.shape

torch.Size([512])

In [39]:
mention_index.shape

torch.Size([22, 512])

In [13]:
state_dict = torch.load("../TUCOREGCN_BERT_DialogRE/tucoregcn_pytorch_model.pt", map_location="cuda")
for key in list(state_dict.keys()):
    state_dict[key.replace("tucoregcn_bert.turnAttention.fc.weight", "tucoregcn_bert.turnAttention.out_lin.weight")] = state_dict.pop(key)

In [14]:
print("\n".join(list(state_dict.keys())))

tucoregcn_bert.bert.embeddings.word_embeddings.weight
tucoregcn_bert.bert.embeddings.position_embeddings.weight
tucoregcn_bert.bert.embeddings.token_type_embeddings.weight
tucoregcn_bert.bert.embeddings.speaker_embeddings.weight
tucoregcn_bert.bert.embeddings.LayerNorm.weight
tucoregcn_bert.bert.embeddings.LayerNorm.bias
tucoregcn_bert.bert.encoder.layer.0.attention.self.query.weight
tucoregcn_bert.bert.encoder.layer.0.attention.self.query.bias
tucoregcn_bert.bert.encoder.layer.0.attention.self.key.weight
tucoregcn_bert.bert.encoder.layer.0.attention.self.key.bias
tucoregcn_bert.bert.encoder.layer.0.attention.self.value.weight
tucoregcn_bert.bert.encoder.layer.0.attention.self.value.bias
tucoregcn_bert.bert.encoder.layer.0.attention.output.dense.weight
tucoregcn_bert.bert.encoder.layer.0.attention.output.dense.bias
tucoregcn_bert.bert.encoder.layer.0.attention.output.LayerNorm.weight
tucoregcn_bert.bert.encoder.layer.0.attention.output.LayerNorm.bias
tucoregcn_bert.bert.encoder.layer.0

In [15]:
torch.save(state_dict, "../TUCOREGCN_BERT_DialogRE/tucoregcn_pytorch_model.pt")